In [1]:
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
%matplotlib qt

In [2]:
def camera_calib(filenames,nx=9,ny=6):
    objp = np.zeros((ny*nx,3),np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2) #here we created a nice chessboard
    
    imgpoints = []
    objpoints = []

    for fname in filenames:
        img = cv2.imread(fname)
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

        if ret == True:
            imgpoints.append(corners)
            objpoints.append(objp)

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
    return (ret, mtx, dist, rvecs, tvecs)

In [3]:
def undistort(img, cam_params):
    ret, mtx, dist, rvecs, tvecs = cam_params
    return cv2.undistort(img, mtx, dist, None, mtx) 

        
filenames=glob.glob("camera_cal/calibration*.jpg")
cam_params=camera_calib(filenames)

In [4]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [5]:
def convert_to_gray(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0,255)):
    gray = convert_to_gray(img)

    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return binary_output

def mag_thresh(img, sobel_kernel=3, thresh=(0, 255)):
    # Convert to grayscale
    gray = convert_to_gray(img)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= thresh[0]) & (gradmag <= thresh[1])] = 1
    return binary_output

# Define a function to threshold an image for a given range and Sobel kernel
def dir_threshold(img, sobel_kernel=9, thresh=(0, np.pi/2)):
    # Grayscale
    gray = convert_to_gray(img)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

def Canny_threshold(img, thresh=(0,255)):
    # Grayscale
    gray = convert_to_gray(img)
    binary_output = cv2.Canny(gray,thresh[0],thresh[1])
    binary_output[binary_output==255]=1
    return binary_output
    
def hls_threshold(img, thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # s channel is the best for this threshold
    s_channel = hls[:,:,2]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    return binary_output

def combine_Sobel(img, SobelX=True, SobelY=True, SobelX_params=[], SobelY_params=[]):
    gray = convert_to_gray(img)
    binary_output=np.ones_like(gray)
    
    if(SobelX):
        img, sobel_kernel, thresh = SobelX_params
        binary_output = binary_output & abs_sobel_thresh(img=img, orient='x', 
                                                         sobel_kernel=sobel_kernel, thresh=thresh)
    
    if(SobelX):
        img, sobel_kernel, thresh = SobelY_params
        binary_output = binary_output & abs_sobel_thresh(img=img, orient='y', 
                                                         sobel_kernel=sobel_kernel, thresh=thresh)
        
    return binary_output    

def combine_MagDir(img, Mag=True, Dir=True, Mag_params=[], Dir_params=[]):
    gray = convert_to_gray(img)
    binary_output=np.ones_like(gray)
    
    if(Mag):
        img, sobel_kernel, thresh = Mag_params
        binary_output = binary_output & abs_sobel_thresh(img=img, orient='x', 
                                                         sobel_kernel=sobel_kernel, thresh=thresh)
    
    if(Dir):
        img, sobel_kernel, thresh = Dir_params
        binary_output = binary_output & abs_sobel_thresh(img=img, orient='y', 
                                                         sobel_kernel=sobel_kernel, thresh=thresh)
    
    return binary_output

In [6]:
def combine_threshold(img, Canny=True, Sobel=True, MagDir=True, HLS=True,
                     Canny_params=[], Sobel_params=[], MagDir_params=[], HLS_params=[]):

    #only for resizing
    gray = convert_to_gray(img)  
    binary_output=np.zeros_like(gray)
        
    if(Canny):
        img, thresh = Canny_params
        binary_output = binary_output | Canny_threshold(img, thresh)
        canny_img = Canny_threshold(img,thresh)
        
    if(Sobel):
        img, SobelX, SobelY, SobelX_params, SobelY_params = Sobel_params
        binary_output = binary_output | combine_Sobel(img, SobelX, SobelY, SobelX_params, SobelY_params)
        
    if(MagDir):
        img, Mag, Dir, Mag_params, Dir_params = MagDir_params
        binary_output = binary_output | combine_MagDir(img, Mag, Dir, Mag_params, Dir_params)
        
    if(HLS):
        img, thresh = HLS_params
        binary_output = binary_output | hls_threshold(img,thresh)
        hls_img = hls_threshold(img,thresh)

    return binary_output

In [7]:
def get_perspective_transform(src = np.float32([[373,605], [923,605], [532, 496], [756,496]]),
                             dst = np.float32([[373,605], [923,605], [373, 487], [923,487]])):
    return cv2.getPerspectiveTransform(src,dst)

def perspective_transform(img,M):
    img_size = (img.shape[1], img.shape[0])
    return cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

In [8]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def mask_image(img):
    imshape = img.shape
    left_bottom = (100, imshape[0])
    right_bottom = (1250, imshape[0])
    left_top = (660, 410) 
    right_top = (640, 410)
    in_left_bottom = (310, imshape[0])
    in_right_bottom = (1150, imshape[0])
    in_left_top = (700,480)
    in_right_top = (650,480)
    vertices = np.array([[left_bottom, left_top, right_top, right_bottom, in_right_bottom,
                          in_left_top, in_right_top, in_left_bottom]], dtype=np.int32)
    # Masked area
    return region_of_interest(img, vertices)

In [9]:
def window_mask(width, height, img_ref, center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

def find_window_centroids(warped, window_width, window_height, margin):
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    
    # Sum quarter bottom of image to get slice, could use a different ratio
    l_sum = np.sum(warped[int(3*warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    r_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(warped.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(warped.shape[1]/2)
    
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(warped.shape[0]/window_height)):
       # convolve the window into the vertical slice of the image
        image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
        conv_signal = np.convolve(window, image_layer)
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin,warped.shape[1]))
        l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
        # Find the best right centroid by using past right center as a reference
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin,warped.shape[1]))
        r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
        # Add what we found for that layer
        window_centroids.append((l_center,r_center))

    return window_centroids


def draw_window_centroids(warped, window_width, window_height, margin):
    window_centroids = find_window_centroids(warped, window_width, window_height, margin)

    # If we found any window centers
    if len(window_centroids) > 0:

        # Points used to draw all the left and right windows
        l_points = np.zeros_like(warped)
        r_points = np.zeros_like(warped)

        # Go through each level and draw the windows 	
        for level in range(0,len(window_centroids)):
            # Window_mask is a function to draw window areas
            l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
            r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
            # Add graphic points from window mask here to total pixels found 
            l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
            r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

        # Draw the results
        template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
        zero_channel = np.zeros_like(template) # create a zero color channel
        template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
        warpage = np.array(cv2.merge((warped,warped,warped)),np.uint8) # making the original road pixels 3 color channels
        output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results
 
    # If no window centers found, just display orginal road image
    else:
        output = np.array(cv2.merge((warped,warped,warped)),np.uint8)

    print(warped.shape)
    print(output.shape)
    result = cv2.addWeighted(warped, 1, convert_to_gray(output), 0.1, 0)



In [137]:
def fit_lanes(window_centroids, ploty, diff_threshold_left = 6400., diff_threshold_right = 6400.):
    unzipped = zip(*window_centroids)
    leftx, rightx = unzipped

    # Fit a second order polynomial to pixel positions in each fake lane line
    left_fit = np.polyfit(ploty, leftx, 2)
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fit = np.polyfit(ploty, rightx, 2)
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # reality checks
    if( last_line.not_first ):
        if( np.linalg.norm( left_fitx - last_line.left_fitx ) > diff_threshold_left  ):
            print("OK sub left")
            left_fitx = last_line.left_fitx

    if( last_line.not_first ):
        last_line.left_fitx = (left_fitx + 2*last_line.left_fitx)/3.
    else:
        last_line.left_fitx = left_fitx

    if( last_line.not_first ):
        if( np.linalg.norm( right_fitx - last_line.right_fitx ) > diff_threshold_right ):
            print("OK sub right")
            right_fitx = last_line.right_fitx
    
    if( last_line.not_first ):
        last_line.right_fitx = (right_fitx + 2*last_line.right_fitx)/3.
    else:
        last_line.right_fitx = right_fitx
        
    last_line.not_first = True
    
    return left_fitx, right_fitx

In [11]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        self.not_first = False
        self.left_fitx = []
        self.right_fitx = []
        
last_line = Line()

In [12]:
def draw_lines(warped, left_fitx, right_fitx, Minv, undist, ploty):

    ploty_inv = 720-ploty
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty_inv]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty_inv])))])
    pts = np.hstack((pts_left, pts_right))

    imshape=warped.shape
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = perspective_transform(color_warp, Minv) 

    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    return result

In [13]:
def curvature(leftx, rightx, ploty):
    y_eval = np.max(ploty)

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    print(left_curverad, 'm', right_curverad, 'm')
    # Example values: 632.1 m    626.2 m

In [149]:
def process_image(original):
    
    undistorted = undistort(original, cam_params)
    
    Canny_params = undistorted, (50,150)
    SobelX_params = undistorted, 7, (10,255)
    SobelY_params = undistorted, 7, (10,255)
    Sobel_params = undistorted, True, True, SobelX_params, SobelY_params
    Mag_params = undistorted, 7, (0,255)
    Dir_params = undistorted, 7, (0.65, 1.05)
    MagDir_params = undistorted, True, True, Mag_params, Dir_params
    HLS_params = undistorted, (70,256)


    thresholded = combine_threshold(undistorted, Canny=True, Sobel=True, MagDir=False, HLS=True,
                                   Canny_params=Canny_params, Sobel_params=Sobel_params, 
                                    MagDir_params=MagDir_params, HLS_params=HLS_params)

    #masked = mask_image(thresholded)
    masked = thresholded
    M=get_perspective_transform()
    Minv=np.linalg.inv(M)
    warped = perspective_transform(masked,M)

    # window settings
    window_width = 50 
    window_height = 80 # Break image into 9 vertical layers since image height is 720
    margin = 100 # How much to slide left and right for searching
    draw_window_centroids(warped, window_width, window_height, margin)

    centroids = find_window_centroids(warped, window_width, window_height, margin)

    ploty = np.linspace(0, 719, num=9)
    leftx, rightx = fit_lanes(centroids, ploty, 600, 200)

    result = draw_lines(warped,leftx, rightx, Minv, undistorted, ploty)

    print(curvature(leftx, rightx, ploty))
    return result
    

In [150]:
#original = cv2.imread("test_images/test2.jpg")



from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Let's try the one with the solid white lane on the right first ...

# In[8]:

last_line.not_first = False

white_output = 'output.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
get_ipython().magic('time white_clip.write_videofile(white_output, audio=False)')




# Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

# In[ ]:



(720, 1280)
(720, 1280, 3)
277.826890971 m 367.361344103 m
None
[MoviePy] >>>> Building video output.mp4
[MoviePy] Writing video output.mp4


  0%|          | 0/1261 [00:00<?, ?it/s]

(720, 1280)
(720, 1280, 3)


  0%|          | 1/1261 [00:00<03:49,  5.50it/s]

277.826890971 m 367.361344103 m
None
(720, 1280)
(720, 1280, 3)


...

100%|█████████▉| 1259/1261 [05:44<00:00,  3.48it/s]

(720, 1280)
(720, 1280, 3)
5194.5195477 m 1302.08143238 m
None


100%|█████████▉| 1260/1261 [05:44<00:00,  3.69it/s]

(720, 1280)
(720, 1280, 3)
3252.13759155 m 2559.58450774 m
None


[MoviePy] Done.
[MoviePy] >>>> Video ready: output.mp4 

CPU times: user 7min 10s, sys: 15.3 s, total: 7min 25s
Wall time: 5min 47s
